In [7]:
def conductor_section(current, potency)->float:
    if potency == 500:
        return current/3
    if potency >= 500 and potency <= 1000:
        return current/2.5
    if potency >= 1000 and potency <= 3000:
        return current/2
    return 'Invalid value'

In [15]:
def core_section(depth:float, width:float)->float:
    geometric_section = depth*width
    magnetic_section = geometric_section/1.1

    return magnetic_section

In [12]:
def calculate_turns_number(
    tension:float,
    sm:float,
    frequency:float,
    bm:float = 11300
    )->float:

    return (tension/sm)*(1e8/(4.44*bm*frequency))

In [16]:
def exec_possibility(
    first_tension: float,
    second_tension: float,
    first_depth: float,
    first_width: float,
    second_depth: float,
    second_width: float,
    frequency: float,
    bm:float = 11300
    )->bool:
    s1 = core_section(first_depth, first_width)
    s2 = core_section(second_depth, second_width)
    n1 = calculate_turns_number(first_tension, s1, frequency, bm)
    n2 = calculate_turns_number(second_tension, s2, frequency, bm)
    section_copper = n1*s1 + n2*s2
    section_window =

    if (section_window/section_copper) < 3:
        print('Impossible to project thistransformer.\
        Please, reproject the transformer with a bigger core')
        return False
    return True

SyntaxError: invalid syntax (<ipython-input-16-f18af69998a4>, line 2)

In [ ]:
def transformador_monofasico(v_prim, v_sec, pot):
    return num_espiras, bitolas, lamina, dim_transformador